<a href="https://colab.research.google.com/github/Muhammad-Murtaz/LLM-Fine-tuning/blob/main/LLAMA_FINE_TUNING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q unsloth trl peft accelerate bitsandbytes datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.1/381.1 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 117.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
max_seq_length = 2048 # max prompt + response length


model, tokenizer = FastLanguageModel.from_pretrained(
model_name = "unsloth/llama-3-8b-bnb-4bit", # 4-bit quantized Llama 3
max_seq_length = max_seq_length,
dtype = None, # auto-detect
load_in_4bit = True, # critical for Colab GPU
)

==((====))==  Unsloth 2026.1.2: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [35]:
dataset = load_dataset("yahma/alpaca-cleaned", split="train")

In [36]:
dataset = dataset.select(range(5000))

In [37]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.


### Instruction:
{instruction}


### Input:
{input}


### Response:
{output}"""

In [38]:
EOS_TOKEN = tokenizer.eos_token


def format_prompts_func(examples):
  texts = []
  for inst, inp, out in zip(examples["instruction"], examples["input"], examples["output"]):
    text = alpaca_prompt.format(
    instruction = inst,
    input = inp,
    output = out,
    ) + EOS_TOKEN
    texts.append(text)
  return {"text": texts}


dataset = dataset.map(format_prompts_func, batched=True)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [39]:
model = FastLanguageModel.get_peft_model(
model,
r = 16, # LoRA rank
lora_alpha = 16,
lora_dropout = 0.0,
bias = "none",
target_modules = [
"q_proj", "k_proj", "v_proj",
"o_proj", "gate_proj", "up_proj", "down_proj",
],
use_gradient_checkpointing = "unsloth",
random_state = 42,
)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [40]:
trainer = SFTTrainer(
model = model,
tokenizer = tokenizer,
train_dataset = dataset,
dataset_text_field = "text",
max_seq_length = max_seq_length,
args = TrainingArguments(
output_dir = "./llama3-unsloth",
per_device_train_batch_size = 2,
gradient_accumulation_steps = 4,
warmup_steps = 10,
num_train_epochs = 1,
learning_rate = 2e-4,
fp16 = not torch.cuda.is_bf16_supported(),
bf16 = torch.cuda.is_bf16_supported(),
logging_steps = 10,
optim = "adamw_8bit",
weight_decay = 0.01,
lr_scheduler_type = "linear",
seed = 42,
save_strategy = "epoch",
report_to = "none",
),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/5000 [00:00<?, ? examples/s]

In [41]:
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,000 | Num Epochs = 1 | Total steps = 625
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Step,Training Loss
10,0.909800
20,0.937100
30,0.981700
40,0.874900
50,0.961100
60,0.936500
70,0.972400
80,0.909100
90,0.962200
100,0.901800


TrainOutput(global_step=625, training_loss=0.9067909385681152, metrics={'train_runtime': 4179.9483, 'train_samples_per_second': 1.196, 'train_steps_per_second': 0.15, 'total_flos': 5.746764675357082e+16, 'train_loss': 0.9067909385681152, 'epoch': 1.0})

In [42]:
FastLanguageModel.for_inference(model)


prompt = """### Instruction:
Explain the legal requirements for starting a tech startup.


### Response:
"""


inputs = tokenizer(
[prompt],
return_tensors="pt",
).to("cuda")

outputs = model.generate(
**inputs,
max_new_tokens = 256,
do_sample = True,
temperature = 0.7,
top_p = 0.9,
)


print(tokenizer.decode(outputs[0], skip_special_tokens=True))

### Instruction:
Explain the legal requirements for starting a tech startup.


### Response:
Starting a tech startup involves several legal requirements that must be met to ensure compliance with the law and protect the interests of the business and its founders. Here are some of the legal requirements for starting a tech startup:

1. Business Structure: One of the first steps in starting a tech startup is choosing a business structure. This could be a sole proprietorship, partnership, limited liability company (LLC), or corporation. Each structure has its own advantages and disadvantages, and the choice will depend on the specific needs of the business.

2. Business Name: Choosing a business name is another important step in starting a tech startup. The business name must be unique and not already in use by another business. It is also important to check if the business name is available as a domain name and social media handles.

3. Business Registration: Once the business structure 